In [ ]:
#CAPACITY.PY - this script was created to facilitate the gathering of capacity data from disparate sources

import json
import vertica_python
import locale
import urllib
import requests
import collections
import logging
import urllib3
import pandas as pd
from urllib.request import urlopen
from requests.auth import HTTPBasicAuth
from pprint import pprint
from datetime import date,timedelta,datetime

conn_info = {'host':'ip-address',
'port':8888,
'user':'username',
'password':'password',
'database':'dbname',
'read_timeout':10}

xvratio = (1.11)
lcratio = (2.22)
caratio = (3.33)
xaratio = (4.44)
xfratio = (5.55)

delivery = ["role1", "role2", "role3"]
datacenter = ["xv", "lc", "ca", "xa", "xf"]
dcratiodict = {"xv":xvratio, "lc":lcratio, "ca":caratio, "xa":xaratio, "xf":xfratio}

today = date.today()
print(today)

def capacity():
    def getadreqvolume():
    
        thisweek = date.today()
        lastweek = date.today() - timedelta(7)
    
        start_date = lastweek.strftime('%Y%m%d')
        end_date = thisweek.strftime('%Y%m%d')
        today = str(date.today())
    
        connection = vertica_python.connect(**conn_info)
        print('Connection successful')
    
        cur = connection.cursor('dict')
        cur.execute("SELECT columns AS tot_req \
            FROM tables \
            WHERE criteria \
            GROUP BY blank ORDER BY tot_req".format(start_date, end_date))
        rows = cur.fetchall()
    
        for row in rows:
            hivol = row['tot_req']
            hivoldate = row['utc_date_sid']
            print("Date with highest volume is", hivoldate, "with", hivol, "requests.")
    
            cur.execute("SELECT columns AS tot_req \
                FROM tables \
                WHERE criteria \
                GROUP BY blank \
                ORDER BY 1,2".format(hivoldate))
            volrows = cur.fetchall()
            
            for vol in volrows:
                pprint(vol)
    
    
        connection.close()
    
    getadreqvolume()
    
    
    def deliveryfn():
        try:
            for dc in datacenter:
                for role in delivery:
                    url = "url.com"
        
                    response = urlopen(url).read().decode()
                    hostobj = json.loads(response)[0]['datapoints']
                    df = pd.DataFrame(hostobj)
        
                    s = df.iloc[0,0]
        
                    capacity = s*86400/1.4*.7
        
                    print(str(dc) + " " + str(role) + " capacity: " + str(capacity))
        except Exception as e:
            logging.exception("message")
            
    deliveryfn()
    
    def datasysfn():
        try:
            for k, v in dcratiodict.items():
                api_url = "api-url.com"
                api_auth = HTTPBasicAuth('username', 'password')
                api_headers = {'stuff'}
                
                urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
                response = requests.get(api_url, headers=api_headers, auth=api_auth, verify=False)
                nodes = response.json()
                
                datasyscount = int(len(nodes['list']))
                calculatedmax = datasyscount * 2000 / float(v) * 86400
                
                print(str(k) + " riak uds capacity: " + str(calculatedmax))
        except Exception as e:
            logging.exception("message")
                    
    datasysfn()
    
    def datafn():
        try:
            api_url = "api-url2"
            api_auth = HTTPBasicAuth('username', 'password')
            api_headers = {'stuff'}
                
            urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(api_url, headers=api_headers, auth=api_auth, verify=False)
            nodes = response.json()
            
            gridcount = int(len(nodes['list']))
            print("global grid capacity: " + str(gridcount * 29450000))
    
        except Exception as e:
            logging.exception("message")
        
    datafn()

if __name__ == '__main__':
    capacity()